In [14]:
import pyarrow.dataset as ds
import duckdb
import plotly.express as px
import json

from plotly.graph_objs import Figure

INDICADORES = {
    "Adequação da Formação Docente": "AFD",
    "Complexidade de Gestão da Escola": "ICG",
    "Esforço Docente": "IED",
    "Média de Alunos por Turma": "ATU",
    "Média de Horas-aula diária": "HAD",
    "Percentual de Docentes com Curso Superior": "DSU",
    "Taxas de Distorção Idade-série": "TDI"
}

DIMENSIONS = {
    "Região geográfica": "NO_REGIAO",
    "Sigla da Unidade da Federeção": "SG_UF",
    "Unidade da Federação": "NO_UF",
    "Mesorregião": "NO_MESORREGIAO",
    "Microrregião": "NO_MICRORREGIAO",
    "Município": "NO_MUNICIPIO"
}

con = duckdb.connect('../file.db')

microdados = ds.dataset("../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

indicadores = {}
for indicador in INDICADORES.values():
    indicadores[indicador] = ds.dataset(f"../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
    con.register(indicador, indicadores[indicador])

In [3]:
DEPENDENCIAS = ['Estadual', 'Municipal', 'Privada', 'Federal']
CATEGORIAS = ["Urbana", "Rural"]
DIMENSOES_GEOGRAFICAS = {
    "Unidade da Federação": ["SG_UF", "uf", "UF_05"],
    "Município": ["NO_MUNICIPIO", "municipio", "NOME"]
}

In [15]:
df = con.execute(f"""
                select
                    DISTINCT(TP_GRUPO)
                from DSU
            """
).df()

In [16]:
df

,TP_GRUPO
0,Ensino Fundamental Anos iniciais
1,Educação de Jovens e Adultos (EJA)
2,Educação Infantil
3,Educação Especial
4,Educação Infantil Pré-Escola
5,Ensino Fundamental Anos finais
6,Educação Infantil Creche
7,EDU_BAS_CAT_0
8,Ensino Fundamental
9,PROF_CAT_0


In [6]:
indicador = "Complexidade de Gestão da Escola"
dimensao_geografica = "Unidade da Federação"
categoria = "Rural"
dependencia = "Estadual"
grupo = "Ensino Fundamental"
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    {DIMENSOES_GEOGRAFICAS[dimensao_geografica][0]} as '{dimensao_geografica}',
                    avg(METRICA) AS '{indicador}'
                from {INDICADORES[indicador]} i
                where
                    NO_CATEGORIA='{categoria}'
                    and NO_DEPENDENCIA='{dependencia}'
                    and TP_GRUPO='{grupo}'
                group by NU_ANO_CENSO, {DIMENSOES_GEOGRAFICAS[dimensao_geografica][0]}
                order by 1, 2
            """
).df()

BinderException: Binder Error: No function matches the given name and argument types 'avg(VARCHAR)'. You might need to add explicit type casts.
	Candidate functions:
	avg(DECIMAL) -> DECIMAL
	avg(SMALLINT) -> DOUBLE
	avg(INTEGER) -> DOUBLE
	avg(BIGINT) -> DOUBLE
	avg(HUGEINT) -> DOUBLE
	avg(DOUBLE) -> DOUBLE

LINE 5:                     avg(METRICA) AS 'Complexidade de Gestão da Escola'
                from ICG i
                where
                    NO_CATEGORIA='Rural'
                    and NO_DEPENDENCIA='Estadual'
                    and TP_GRUPO='Ensino Fundamental'
                group by NU_ANO_CENSO, SG_UF
                order by 1, 2
            ...
                            ^

In [57]:
df.info()

In [46]:
grupos = con.execute(f"select distinct(TP_GRUPO) as value from {INDICADORES[indicador]} limit 10").df()

In [48]:
a = grupos.value

In [50]:
a.tolist()

['Ensino Fundamental - 3º Ano',
 'Ensino Fundamental - 8º Ano',
 'Ensino Médio - 2º Ano',
 'Ensino Fundamental',
 'Ensino Fundamental - 5º Ano',
 'Ensino Fundamental Anos finais',
 'Ensino Fundamental - 1º Ano',
 'Ensino Fundamental - 4º Ano',
 'Ensino Fundamental - 9º Ano',
 'Ensino Médio - 3º Ano']

In [54]:
df[ df["Unidade da Federação"].isin(["AC"])]

In [17]:
with open(f"../data/geo/{DIMENSOES_GEOGRAFICAS[dimensao_geografica][1]}.json", encoding="latin1") as f:
    geojson = json.load(f)

In [21]:
fig = px.choropleth_mapbox(
    df,
    geojson=geojson,
    color=INDICADORES[indicador],
    locations=dimensao_geografica,
    range_color=(0, 100),
    mapbox_style = "white-bg",
    featureidkey=f"properties.{DIMENSOES_GEOGRAFICAS[dimensao_geografica][2]}",
    center = {"lat": -14, "lon": -55},
    animation_frame="Ano",
    zoom=3,
    width=750,
    height=750
)

fig.write_html(f"plots/{indicador}_{dimensao_geografica}.html")